In [1]:
import numpy as np
import json
import zipfile
import bz2
import time
import pandas as pd

In [8]:
print("Load data")
with zipfile.ZipFile('./experiments/linear-VAR_N-3_T-150.zip', "r") as zip_ref:
    for name in sorted(zip_ref.namelist()):

        data = np.loadtxt(zip_ref.open(name))

        data_df = pd.DataFrame(data)

        import sys
        sys.path.append("..")

        from d2c.simulatedTimeSeries import SimulatedTimeSeries
        from d2c.D2C import D2C

        d2c_test = D2C([None],[data_df])
        X_test = d2c_test.compute_descriptors_no_dags()

        training_data = pd.read_csv('./timeseries_training.csv')

        X_train = training_data.drop(['graph_id', 'edge_source', 'edge_dest', 'is_causal'], axis=1)
        y_train = training_data['is_causal']

        from sklearn.ensemble import RandomForestClassifier
        test_df = pd.DataFrame(X_test).drop(['graph_id', 'edge_source', 'edge_dest'], axis=1)
        clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(test_df)

        returned = pd.concat([X_test, pd.DataFrame(y_pred, columns=['is_causal'])], axis=1)


        break

Load data
17:49:52 Descriptors for DAG 0 edge pair 0 1 computed
17:49:53 Descriptors for DAG 0 edge pair 0 2 computed
17:49:54 Descriptors for DAG 0 edge pair 1 0 computed
17:49:55 Descriptors for DAG 0 edge pair 1 2 computed
17:49:56 Descriptors for DAG 0 edge pair 2 0 computed
17:49:57 Descriptors for DAG 0 edge pair 2 1 computed


In [11]:
X_test

[{'graph_id': 0,
  'edge_source': 0,
  'edge_dest': 1,
  'effca': -5.295741012750196e-17,
  'effef': -2.71244643887852e-16,
  'comcau': 0.046588578532125745,
  'delta': 0.0,
  'delta2': 0.0,
  'delta.i1': 0.0,
  'delta.i2': 0.0,
  'delta.i3': 0.0,
  'delta.i4': 0.0,
  'delta.i5': 0.0,
  'delta.i6': 0.0,
  'delta.i7': 0.0,
  'delta2.i1': 0.0,
  'delta2.i2': 0.0,
  'delta2.i3': 0.0,
  'delta2.i4': 0.0,
  'delta2.i5': 0.0,
  'delta2.i6': 0.0,
  'delta2.i7': 0.0,
  'ca.ef': 0,
  'ef.ca': 0,
  'I1.i1': 0.0,
  'I1.i2': 0.0,
  'I1.i3': 0.0,
  'I1.i4': 0.0,
  'I1.i5': 0.0,
  'I1.i6': 0.0,
  'I1.i7': 0.0,
  'I1.j1': 0.0,
  'I1.j2': 0.0,
  'I1.j3': 0.0,
  'I1.j4': 0.0,
  'I1.j5': 0.0,
  'I1.j6': 0.0,
  'I1.j7': 0.0,
  'I2.i1': 0.0,
  'I2.i2': 0.0,
  'I2.i3': 0.0,
  'I2.i4': 0.0,
  'I2.i5': 0.0,
  'I2.i6': 0.0,
  'I2.i7': 0.0,
  'I2.j1': 0.0,
  'I2.j2': 0.0,
  'I2.j3': 0.0,
  'I2.j4': 0.0,
  'I2.j5': 0.0,
  'I2.j6': 0.0,
  'I2.j7': 0.0,
  'I3.i1': 0.0,
  'I3.i2': 0.0,
  'I3.i3': 0.0,
  'I3.i4': 0

In [15]:
returned = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_pred, columns=['is_causal'])], axis=1)
of_interest = returned[['edge_source', 'edge_dest','is_causal']]
of_interest.to_clipboard()


In [16]:
val_matrix = np.zeros((3, 3), dtype='float32')
for index, row in of_interest.iterrows():
    source = row['edge_source']
    dest = row['edge_dest']
    weight = row['is_causal']
    val_matrix[source, dest] = weight

print(val_matrix)

[[0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]]
